In [ ]:
# import required libraries

import pandas as pd
import numpy as np
import requests as rq
import json
import logging
import os

### Get the Session ID

In [ ]:
# Take creadential username and password from other notebook file

#Reading credentials & URL from text file
file= open('./credentials.txt')

content = file.read()
#print(content)
file.close()

read_file_content = dict(subString.split("=") for subString in content.split(";"))
#print(read_file_content)

# qmsRecords = read_file_content['qmsRecords'].split()
# print(qmsRecords)

#Defining URL
auth_URL = f"{read_file_content['vaultDNS']}api{read_file_content['version_Api']}auth"

#Defining Body
auth_body = {
    'username': read_file_content['username'],
    'password': read_file_content['password']
}

#Defining Header
auth_headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'Accept': 'application/json'
}

auth_response = rq.post(auth_URL, headers = auth_headers, data = auth_body)


auth_response_data = json.loads(auth_response.content)

sessionId=''

if auth_response_data['responseStatus'] == 'SUCCESS':
    sessionId = auth_response_data['sessionId']
    print('sessionId stored in "sessionId variable"')
    print('Response Status Below :')
    print(auth_response_data['responseStatus'])
    #print('sessionId Below :')
    #print(sessionId)
else:
    print('something went wrong')
    print(auth_response_data['responseStatus'])
    print(auth_response_data['responseMessage'])

### Read the CSV file QMS record with attachment.
Report should be configured in vault - QMS record with attachments.
Csv file must contain Attachment Id, Record Id, Record Name atleast. Good to have format and size also included for verification.

In [ ]:
attachments_for_qms_record = pd.read_csv('csvFile_QMSrecordWithAttachment_filecontainsAttachmentIDs')
attachments_for_qms_record.head()

In [ ]:
attachments_for_qms_record.shape

### Script to download the FOR from the QMS record attachment section

In [ ]:
dwnld_FOR_attach_header = {
    'Authorization': sessionId,
    'Accept': 'application/json'
}

obj_name = 'Object_api_name'  #for Example - quality_event_qdm

for i in attachments_for_qms_record.attach_ID:   #attach_ID is the column here

    #taking the QMS record ID and Qms record name from the csv sheet
    record_id = attachments_for_qms_record.id[attachments_for_qms_record['attach_ID'] == i].values[0]
    record_name = attachments_for_qms_record.Name[attachments_for_qms_record['id'] == record_id].values[0]
    print(f"Attchment ID {i} is associated with {record_name} having record id {record_id}")
    #print(record_name)
    
    
    # Defining Object Attachment URL Path
    dwnld_FOR_attch_url = f"{read_file_content['vaultDNS']}api{read_file_content['version_Api']}vobjects/{obj_name}/{record_id}/attachments/{i}/file/"
    #print(dwnld_attch_url)
    
    dwnld_FOR_attach_response = rq.get(dwnld_FOR_attch_url, headers = dwnld_FOR_attach_header)
    
    #COMMENT HERE#
    
    #print('########')
    #print(dwnld_FOR_attach_response.status_code) #This will print the status code
    #print('########')
    #print(dwnld_FOR_attach_response.headers) #This will print the response headers
    #print('########')
    #print(dwnld_FOR_attach_response.content) #This will returns the content of the response in bytes
    #print('########')
    #print(dwnld_FOR_attach_response.text) #This will returns the content of the response, in unicode.
    #print('########')
    
    #COMMENT HERE#
    
    dwnld_FOR_attch_header_string_convert = json.dumps(dict(dwnld_FOR_attach_response.headers))

    dwnld_FOR_attch_header_dict_convert = json.loads(dwnld_FOR_attch_header_string_convert)
    #print (dwnld_FOR_attch_header_dict_convert)
    
    if (('application/zip' or 'application/pdf' in dwnld_FOR_attch_header_dict_convert['Content-Type']) and dwnld_FOR_attach_response.status_code == 200):
     
        #Saving the Zip file
        filename = dwnld_FOR_attch_header_dict_convert['Content-Disposition'].split("''")[1]
        #print(filename)
        
        #filename_id = f"{i}_{filename}".replace('%20', ' ')
        #filename_id = f"{record_name}_{filename}".replace('%20', ' ')
        filename_id = f"{filename}".replace('%20', ' ')
        filename_id = filename_id.replace('%2C', ',')
        filename_id = filename_id.replace('%28', '(')
        filename_id = filename_id.replace('%29', ')')
        filename_id = f"{record_name}.{filename_id.split('.')[2]}"
#         filename_id = filename_id.replace('%C2%B0','°')
#         filename_id = filename_id.replace('%E2%80%93','-')
#         filename_id = filename_id.replace('%C2%AE','®')
#         filename_id = filename_id.replace('%E2%84%A2','™')
#         filename_id = filename_id.replace('%CE%B1','α')
#         filename_id = filename_id.replace('%CE%B2','β')
#         filename_id = filename_id.replace('%CE%B3','γ')
#         filename_id = filename_id.replace('%CE%B4','δ')
#         filename_id = filename_id.replace('%25','%')
#         filename_id = filename_id.replace('%E2%80%99',"'")
#         filename_id = filename_id.replace('%27',"'")

         #Download the filename to local
        directory_path = r'Path to save the Attachments'
        file_path = os.path.join(directory_path, filename_id)             
        with open(file_path, 'wb') as output_file:
                output_file.write(dwnld_FOR_attach_response.content)
                print(f"Attachment ID {i}::{record_name} - FOR downloading completed for QMS Record ID {record_id}")
                
        # Put log into lofg file     
        logger = logging.getLogger(__name__)
        logging.basicConfig(filename="logFile.log",
                    encoding="utf-8",
                    level=logging.DEBUG,
                    format='%(asctime)s %(message)s', 
                    datefmt='%m/%d/%Y %I:%M:%S %p')
    
        logger.info('SUCCESS:: FOR attachment id '+ str(i) + ' downloaded successfully for QMS Record '+ record_name +' OR QMS ID ' + str(record_id))
    
        print(f"Remaining FOR downloading API Burst Limit {dwnld_FOR_attch_header_dict_convert['X-VaultAPI-BurstLimitRemaining']}")
        print('###################################################')
        
    else:
        
        print(f"Attachment ID {i}::{record_name} - FOR downloading failed for QMS Record ID {record_id}")
        logger = logging.getLogger(__name__)
        logging.basicConfig(filename="logFile.log",
                    encoding="utf-8",
                    level=logging.DEBUG,
                    format='%(asctime)s %(message)s', 
                    datefmt='%m/%d/%Y %I:%M:%S %p')
        logger.info('WARNING:: FOR attachment id '+ str(i) + ' downloading failed for QMS Record '+ record_name +' OR QMS ID '+ str(record_id) + ' Check manually.')
        print(f"Remaining FOR downloading API Burst Limit {dwnld_FOR_attch_header_dict_convert['X-VaultAPI-BurstLimitRemaining']}")
        print('###################################################')
        
        
print("#################################################################################")
print("###################### TASK COMPLETED  ##########################################")
print("#################################################################################")